In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, trim, udf
from pyspark.sql.types import FloatType
import re

# ścieżka do bazy danych hurtowni danych oraz plików
# należy dostosować do ścieżki względnej, w której umieszczony został bieżący notebook
warehouse_location = '/opt/spark/work-dir/Lab7/metastore_db'

# utworzenie sesji Spark, ze wskazaniem włączenia obsługi Hive oraz
# lokalizacją przechowywania hurtowni danych
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Apache SQL and Hive")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .enableHiveSupport()\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 19:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/16 19:04:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[2] appName=Apache SQL and Hive>

In [3]:
# Wczytanie pliku TXT
df = spark.read.option("delimiter", ";").option("header", "true").csv("zamowienia.txt")

In [4]:
# Sprawdzenie nazw kolumn
print(df.columns)

['Kraj', 'Sprzedawca', 'Data zamowienia', 'idZamowienia', 'Utarg']


In [5]:
# Naprawienie kodowania w kolumnie Sprzedawca
df = df.withColumn("Sprzedawca", regexp_replace(trim(col("Sprzedawca")), "[^\\x00-\\x7F]", ""))

In [6]:
# Funkcja do konwersji wartości Utarg z tekstu na liczbę float
def convert_utarg(value):
    # Usuwamy wszystkie niecyfrowe znaki poza przecinkiem i kropką
    value = re.sub(r"[^\d,\.]", "", value)
    # Zamieniamy przecinek na kropkę (dla konwersji na float)
    value = value.replace(",", ".")
    try:
        return float(value)
    except ValueError:
        return None

In [7]:
# Rejestrujemy funkcję UDF do użycia w Spark SQL
convert_utarg_udf = udf(convert_utarg, FloatType())

In [8]:
# Poprawienie formatu kolumny Utarg
df = df.withColumn("Utarg", convert_utarg_udf(col("Utarg")))

In [9]:
# Konwersja kolumny idZamowienia na typ Integer
df = df.withColumn("idZamowienia", col("idZamowienia").cast("int"))

In [10]:
# Wyświetlenie przykładowych danych
df.show(10)

[Stage 1:>                                                          (0 + 1) / 1]

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     16.07.2003|       10248| 440.0|
|Polska|   Sowiski|     10.07.2003|       10249|1863.4|
|Niemcy|   Peacock|     12.07.2003|       10250|1552.6|
|Niemcy| Leverling|     15.07.2003|       10251|654.06|
|Niemcy|   Peacock|     11.07.2003|       10252|3597.9|
|Niemcy| Leverling|     16.07.2003|       10253|1444.8|
|Polska|  Kowalski|     23.07.2003|       10254|556.62|
|Polska|     Dudek|     15.07.2003|       10255|2490.5|
|Niemcy| Leverling|     17.07.2003|       10256| 517.8|
|Niemcy|   Peacock|     22.07.2003|       10257|1119.9|
+------+----------+---------------+------------+------+
only showing top 10 rows



In [11]:
# Wiaderkowanie danych wg. kolumny 'Sprzedawca' z 4 wiadrami
df.write.bucketBy(4, "Sprzedawca").sortBy("Sprzedawca").saveAsTable("bucketed_orders")

24/12/16 19:04:15 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/16 19:04:15 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/16 19:04:19 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/16 19:04:19 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.2
24/12/16 19:04:19 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/12/16 19:04:22 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/12/16 19:04:22 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/12/16 19:04:22 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/16 19:04:22 WARN 

In [12]:
!ls metastore_db/bucketed_orders/*.parquet

metastore_db/bucketed_orders/part-00000-c147e246-3d34-407f-8d98-18b217830b52_00000.c000.snappy.parquet
metastore_db/bucketed_orders/part-00000-c147e246-3d34-407f-8d98-18b217830b52_00001.c000.snappy.parquet
metastore_db/bucketed_orders/part-00000-c147e246-3d34-407f-8d98-18b217830b52_00003.c000.snappy.parquet


In [13]:
from pyspark.sql import functions as F
import time

# Czas zapytania agregującego na danych z wiaderkami
start_time = time.time()
df_bucketed = spark.table("bucketed_orders")
df_bucketed.filter(df_bucketed["Sprzedawca"] == "Kowalski").groupby("Sprzedawca").agg(F.avg("Utarg")).show()
bucketed_time = time.time() - start_time

+----------+-----------------+
|Sprzedawca|       avg(Utarg)|
+----------+-----------------+
|  Kowalski|1637.910719916934|
+----------+-----------------+



In [14]:
start_time = time.time()
df.filter(df["Sprzedawca"] == "Kowalski").groupby("Sprzedawca").agg(F.avg("Utarg")).show()
non_bucketed_time = time.time() - start_time

+----------+-----------------+
|Sprzedawca|       avg(Utarg)|
+----------+-----------------+
|  Kowalski|1637.910719916934|
+----------+-----------------+



In [15]:
print(f"Czas z wiaderkami: {bucketed_time} sek.")
print(f"Czas bez wiaderków: {non_bucketed_time} sek.")

Czas z wiaderkami: 1.3030173778533936 sek.
Czas bez wiaderków: 1.0304524898529053 sek.


In [16]:
# Partycjonowanie danych wg. kraju i nazwiska
df.write.partitionBy("Kraj", "Sprzedawca").csv("data_to_save", header=True)

In [17]:
# Zapytanie agregujące na danych oryginalnych
start_time = time.time()
df.filter(df["Kraj"] == "Polska").groupby("Sprzedawca").agg(F.avg("Utarg")).show()
original_time = time.time() - start_time

+----------+------------------+
|Sprzedawca|        avg(Utarg)|
+----------+------------------+
|      King|1745.7162600844654|
|     Dudek|1830.4399899738592|
|   Sowiski|1115.8096831688515|
|  Kowalski| 1637.910719916934|
+----------+------------------+



In [18]:
# Załadowanie partycjonowanych danych
df_partitioned = spark.read.option("delimiter", ",").csv("data_to_save", header=True, inferSchema=True)

In [19]:
# Zapytanie agregujące na danych partycjonowanych
start_time = time.time()
df_partitioned.filter(df_partitioned["Kraj"] == "Polska").groupby("Sprzedawca").agg(F.avg("Utarg")).show()
partitioned_time = time.time() - start_time

+----------+------------------+
|Sprzedawca|        avg(Utarg)|
+----------+------------------+
|      King|1745.7162686567167|
|   Sowiski|1115.8096923076926|
|     Dudek|1830.4399999999998|
|  Kowalski|1637.9107142857142|
+----------+------------------+



In [20]:
# Porównanie czasów
print(f"Czas na danych oryginalnych: {original_time} sek.")
print(f"Czas na danych partycjonowanych: {partitioned_time} sek.")

Czas na danych oryginalnych: 0.358806848526001 sek.
Czas na danych partycjonowanych: 0.4568946361541748 sek.
